In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt
%cd yolov5

import torch
from IPython.display import Image, clear_output
#from utils.google_utils import gdrive_download

clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/yolov5/dataset
!curl -L "https://public.roboflow.com/ds/3a7YoghK5M?key=sSXWEm5AKp" > roboflow.zip; 

/content/drive/MyDrive/yolov5/dataset
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   888  100   888    0     0   2259      0 --:--:-- --:--:-- --:--:--  2259
 38 6262M   38 2415M    0     0  52.7M      0  0:01:58  0:00:45  0:01:13 58.7M^C


In [ ]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/yolov5/dataset/roboflow.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/drive/MyDrive/yolov5/dataset/')
   print('File is unzipped in /content/drive/MyDrive/yolov5/dataset folder')

File is unzipped in /content/drive/MyDrive/yolov5/dataset folder


In [ ]:
%cd /content/drive/MyDrive/yolov5/dataset/

!mkdir train/
!mkdir train/images
!mkdir train/labels
!mkdir valid/
!mkdir valid/images
!mkdir valid/labels
!mkdir test/
!mkdir test/images
!mkdir test/labels

/content/drive/MyDrive/yolov5/dataset


In [ ]:
import os #provides functions for creating and removing a directory (folder), fetching its contents, changing and identifying the current directory,
import shutil #method in Python is used to copy the content of source file to destination file or directory
%cd /content/drive/MyDrive/yolov5/dataset/
# allImages = glob.glob("/export/images/*.jpg")
allImages = []
for filename in os.listdir("/content/drive/MyDrive/yolov5/dataset/export/images"):
    if filename.endswith(".jpg"):
      allImages.append(filename)
trainImages = []
valImages = []
trainLabels = []
valLabels = []
testImages = []
testLabels = []
print(len(allImages)) #total 27000 images

testCount = int(0.2 * len(allImages)/5)
print(testCount) #540
trainCount = len(allImages)/5 - testCount-1
valCount = int(0.2 * trainCount)
trainCount = trainCount - valCount-1

print(trainCount, valCount, testCount)
#1727.0           431         540
# # create training Dataset
j = 0
k = 0
for i in range(0,int(len(allImages)/10)):
  if i < trainCount: # create training images i will go 0 to 2700 & traincount is 1727 
    trainImages.append(allImages[i])
  elif i >= trainCount and i<(valCount + trainCount):
    valImages.append(allImages[i])
  else:
    testImages.append(allImages[i])

print(len(trainImages), len(valImages), len(testImages))

# # moving training images
source = "/content/drive/MyDrive/yolov5/dataset/export/images/"
destination = "train/images/"
for fileT in trainImages:
  shutil.move(source+fileT , destination+fileT)

source = "/content/drive/MyDrive/yolov5/dataset/export/images/"
destination = "valid/images/"
for fileT in valImages:
  shutil.move(source+fileT , destination+fileT)

source = "/content/drive/MyDrive/yolov5/dataset/export/images/"
destination = "test/images/"
for fileT in testImages:
  shutil.move(source+fileT , destination+fileT)

# moving labels
source = "/content/drive/MyDrive/yolov5/dataset/export/labels/"
destination = "train/labels/"
for fileT in trainImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT , destination+fileT)

source = "/content/drive/MyDrive/yolov5/dataset/export/labels/"
destination = "valid/labels/"
for fileT in valImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT , destination+fileT)

source = "/content/drive/MyDrive/yolov5/dataset/export/labels/"
destination = "test/labels/"
for fileT in testImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT , destination+fileT)



/content/drive/MyDrive/yolov5/dataset
28081
561
1796.1 449 561
1797 449 562


In [ ]:
#extracting information from the yaml file of roboflow classes
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 11
names: ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']

In [ ]:
# define number of classes based on data.yaml
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
%cat /content/drive/MyDrive/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
#customize iPython writefile so we can write variables
#A Python module is a text file containing Python code. %%writefile lets you output code developed in a Notebook to a Python module
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/drive/MyDrive/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  
  - [30,61, 62,45, 59,119]  
  - [116,90, 156,198, 373,326] 

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]], 
   [-1, 1, Conv, [128, 3, 2]],  
   #Conv-128-3-1 represents a convolutional layer with 128 filters of kernel size 3 and stride 2
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]], 
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], 
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]], 
  ]

# YOLOv5 head - This is the last layer of YOLOv5, and it detects the results in the form of confidence score, size, and accuracy
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]], 
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  
   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  

   [[17, 20, 23], 1, Detect, [nc, anchors]], 
  ]

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/drive/MyDrive/yolov5
!python train.py --img 416 --batch 16 --epochs 100 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/drive/MyDrive/yolov5
^C
CPU times: user 63.7 ms, sys: 13.8 ms, total: 77.5 ms
Wall time: 7.37 s


In [ ]:
!python /content/yolov5/detect.py --weights /content/drive/MyDrive/yolov5/traineddata/best.pt --img 416 --conf 0.4 --source /content/drive/MyDrive/yolov5/1478020210691558747_jpg.rf.tVGZJvw5LI19xAd0Jzgy.jpg

detect: weights=['/content/drive/MyDrive/yolov5/traineddata/best.pt'], source=/content/drive/MyDrive/yolov5/1478020210691558747_jpg.rf.tVGZJvw5LI19xAd0Jzgy.jpg, data=../../../../yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../../../../yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-164-g177da7f torch 1.11.0+cu113 CPU

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7273488 parameters, 0 gradients
image 1/1 /content/drive/MyDrive/yolov5/1478020210691558747_jpg.rf.tVGZJvw5LI19xAd0Jzgy.jpg: 288x416 2 cars, 1 trafficLight-Green, 1 trafficLight-Red, Done. (0.190s)
Speed: 0.8ms pre-process, 189.8ms inference, 33.1ms NMS per image at shape (1, 3, 416, 416)
Results saved to ../../../